In [22]:
# Dependencies
import pandas as pd
#pd.options.mode.chained_assignment = None  # default='warn'
import sqlite3

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

### Cleaned ARBNB DF Read In

In [23]:
# Store filepath in a variable
ARBNB = "Resources/cleaned_ARBNB.csv"

In [24]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
ARBNB_df = pd.read_csv(ARBNB, encoding="ISO-8859-1")
ARBNB_df.head()

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,...,number_of_reviews_l30d,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,https://www.airbnb.com/rooms/12899,within an hour,100.0,100.0,1,0,Concordia,45.56488,-122.63418,Entire home/apt,...,1,2021-11-30,4.93,4.94,4.98,4.99,4.99,4.93,4.94,4.08
1,https://www.airbnb.com/rooms/37676,within a day,100.0,55.0,1,1,Pearl,45.52564,-122.68273,Entire home/apt,...,1,2021-12-01,4.88,4.86,4.86,4.83,4.77,4.94,4.66,0.91
2,https://www.airbnb.com/rooms/41601,within an hour,100.0,100.0,1,1,Roseway,45.54804,-122.58541,Entire home/apt,...,0,2021-10-13,4.84,4.90,4.90,4.95,4.92,4.67,4.83,1.76
3,https://www.airbnb.com/rooms/61893,within an hour,100.0,73.0,1,1,Goose Hollow,45.52258,-122.69955,Entire home/apt,...,1,2021-12-01,5.00,5.00,5.00,4.93,5.00,5.00,4.93,0.24
4,https://www.airbnb.com/rooms/67036,within a day,100.0,0.0,0,1,Sullivan's Gulch,45.53103,-122.64448,Entire home/apt,...,0,2020-02-27,4.93,5.00,5.00,5.00,5.00,4.79,4.89,0.23


### Cleaned ARBNB Google API DF Read In

In [25]:
# Store filepath in a variable
ARBNB_google_api = "Resources/cleaned_ARBNB_google_api.csv"

In [26]:
ARBNB_google_api = pd.read_csv(ARBNB_google_api, encoding="ISO-8859-1")
ARBNB_google_api.head()

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,...,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,house_number,street_name,zipcode,neighborhood,lat_lng,address
0,https://www.airbnb.com/rooms/12899,within an hour,100.0,100.0,1,0,Concordia,45.56488,-122.63418,Entire home/apt,...,4.99,4.93,4.94,4.08,5827,Northeast 31st Avenue,97211,Concordia,"45.56488, -122.63418",5827 Northeast 31st Avenue
1,https://www.airbnb.com/rooms/37676,within a day,100.0,55.0,1,1,Pearl,45.52564,-122.68273,Entire home/apt,...,4.77,4.94,4.66,0.91,1110,Northwest Flanders Street,97209,Northwest Portland,"45.52564, -122.68273",1110 Northwest Flanders Street
2,https://www.airbnb.com/rooms/41601,within an hour,100.0,100.0,1,1,Roseway,45.54804,-122.58541,Entire home/apt,...,4.92,4.67,4.83,1.76,7510,Northeast Fremont Street,97213,Roseway,"45.54804, -122.58541",7510 Northeast Fremont Street
3,https://www.airbnb.com/rooms/61893,within an hour,100.0,73.0,1,1,Goose Hollow,45.52258,-122.69955,Entire home/apt,...,5.00,5.00,4.93,0.24,2334,Southwest Cactus Drive,97205,Goose Hollow,"45.52258, -122.69955",2334 Southwest Cactus Drive
4,https://www.airbnb.com/rooms/80357,within an hour,100.0,52.0,1,1,Sullivan's Gulch,45.53364,-122.63895,Entire home/apt,...,5.00,5.00,5.00,0.02,2608,Northeast Halsey Street,97232,Sullivan's Gulch,"45.53364, -122.63895",2608 Northeast Halsey Street


In [29]:
#rename some columns
ARBNB_google_api=ARBNB_google_api.rename(columns={"zipcode": "google_zip", "address": "google_address", "neighborhood": "google_neighborhood"})
ARBNB_google_api

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,...,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,house_number,street_name,google_zip,google_neighborhood,lat_lng,google_address
0,https://www.airbnb.com/rooms/12899,within an hour,100.0,100.0,1,0,Concordia,45.564880,-122.634180,Entire home/apt,...,4.99,4.93,4.94,4.08,5827,Northeast 31st Avenue,97211,Concordia,"45.56488, -122.63418",5827 Northeast 31st Avenue
1,https://www.airbnb.com/rooms/37676,within a day,100.0,55.0,1,1,Pearl,45.525640,-122.682730,Entire home/apt,...,4.77,4.94,4.66,0.91,1110,Northwest Flanders Street,97209,Northwest Portland,"45.52564, -122.68273",1110 Northwest Flanders Street
2,https://www.airbnb.com/rooms/41601,within an hour,100.0,100.0,1,1,Roseway,45.548040,-122.585410,Entire home/apt,...,4.92,4.67,4.83,1.76,7510,Northeast Fremont Street,97213,Roseway,"45.54804, -122.58541",7510 Northeast Fremont Street
3,https://www.airbnb.com/rooms/61893,within an hour,100.0,73.0,1,1,Goose Hollow,45.522580,-122.699550,Entire home/apt,...,5.00,5.00,4.93,0.24,2334,Southwest Cactus Drive,97205,Goose Hollow,"45.52258, -122.69955",2334 Southwest Cactus Drive
4,https://www.airbnb.com/rooms/80357,within an hour,100.0,52.0,1,1,Sullivan's Gulch,45.533640,-122.638950,Entire home/apt,...,5.00,5.00,5.00,0.02,2608,Northeast Halsey Street,97232,Sullivan's Gulch,"45.53364, -122.63895",2608 Northeast Halsey Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,https://www.airbnb.com/rooms/53547952,within a day,100.0,83.0,0,1,Brentwood-Darlington,45.471363,-122.606731,Entire home/apt,...,5.00,5.00,5.00,1.00,5421,Southeast Ogden Street,97206,Brentwood-Darlington,"45.47136345619144, -122.60673127640317",5421 Southeast Ogden Street
2033,https://www.airbnb.com/rooms/53623129,within an hour,100.0,100.0,0,1,Pearl,45.529118,-122.680400,Entire home/apt,...,5.00,5.00,5.00,1.00,920,Northwest Kearney Street,97209,Northwest Portland,"45.52911760372456, -122.68039963494245",920 Northwest Kearney Street
2034,https://www.airbnb.com/rooms/53644475,within an hour,100.0,100.0,0,1,Southwest Hills,45.494583,-122.707946,Entire home/apt,...,5.00,5.00,5.00,1.00,3940,Southwest Altadena Avenue,97239,Southwest Portland,"45.494582969682334, -122.70794552446232",3940 Southwest Altadena Avenue
2035,https://www.airbnb.com/rooms/53649962,within an hour,100.0,100.0,0,1,Humboldt,45.555253,-122.666735,Entire home/apt,...,5.00,5.00,5.00,1.00,4335,North Williams Avenue,97217,North Portland,"45.555252918994015, -122.66673548395418",4335 North Williams Avenue


### Cleaned ARBNB Position Stack API DF Read In

In [30]:
# Store filepath in a variable
ARBNB_positionstack_api = "Resources/cleaned_ARBNB_positionstack_api.csv"

In [31]:
ARBNB_positionstack_api = pd.read_csv(ARBNB_positionstack_api, encoding="ISO-8859-1")
ARBNB_positionstack_api.head()

,listing_url,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,room_type,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,address,zipcode,neighborhood,lat_lng
0,https://www.airbnb.com/rooms/12899,within an hour,100.0,100.0,1,0,Concordia,45.56488,-122.63418,Entire home/apt,...,4.98,4.99,4.99,4.93,4.94,4.08,5827 NE 31st Ave,97211.0,Concordia,"45.56488, -122.63418"
1,https://www.airbnb.com/rooms/37676,within a day,100.0,55.0,1,1,Pearl,45.52564,-122.68273,Entire home/apt,...,4.86,4.83,4.77,4.94,4.66,0.91,1110 Northwest Flanders Street,97209.0,Pearl,"45.52564, -122.68273"
2,https://www.airbnb.com/rooms/41601,within an hour,100.0,100.0,1,1,Roseway,45.54804,-122.58541,Entire home/apt,...,4.90,4.95,4.92,4.67,4.83,1.76,7510 NE Fremont St,97213.0,Roseway,"45.54804, -122.58541"
3,https://www.airbnb.com/rooms/61893,within an hour,100.0,73.0,1,1,Goose Hollow,45.52258,-122.69955,Entire home/apt,...,5.00,4.93,5.00,5.00,4.93,0.24,821 SW Green Ave,97205.0,Goose Hollow,"45.52258, -122.69955"
4,https://www.airbnb.com/rooms/67036,within a day,100.0,0.0,0,1,Sullivan's Gulch,45.53103,-122.64448,Entire home/apt,...,5.00,5.00,5.00,4.79,4.89,0.23,1112 NE 21st Ave,97232.0,Sullivan's Gulch,"45.53103, -122.64448"


### Neighborhood DF Read In

In [32]:
# Store filepath in a variable
neighborhood = "Resources/Portland_Zip_and_Neighborhood.csv"

In [33]:
neighborhood_df = pd.read_csv(neighborhood, encoding="ISO-8859-1")
neighborhood_df.head()

,Zipcode,Neighborhood_Name
0,97212,Alameda
1,97217,Arbor Lodge
2,97222,Ardenwald
3,97230,Argay
4,97201,Arlington Heights


### Cleaned Zillow DF Read In

In [34]:
# Store filepath in a variable
zillow = "Resources/cleaned_zillow.csv"

In [35]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
zillow_df = pd.read_csv(zillow, encoding="ISO-8859-1")
zillow_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,1555.0,2021-05-21,315400,175.0,1950.0,273960.0,2020-01-01,https://www.zillow.com/homedetails/2860-NE-Blo...,2007-01-01
1,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,9583.0,2021-06-04,512000,188.0,1995.0,397560.0,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01
2,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
3,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,7000.0,2021-06-01,404200,351.0,1800.0,309260.0,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01
4,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,25.0,SINGLE_FAMILY,45.498184,2036.0,-122.451332,6969.0,2021-05-21,478200,235.0,2250.0,373030.0,2020-01-01,https://www.zillow.com/homedetails/110-NW-Will...,1978-01-01


### Zillow Positionstack DF Read In

In [36]:
# Store filepath in a variable
zillow_positionstack = "Resources/cleaned_zillow_positionstack_api.csv"

In [37]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
zillow_positionstack_df = pd.read_csv(zillow_positionstack, encoding="ISO-8859-1")
zillow_positionstack_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,...,pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt,address,zipcode,neighborhood,lat_lng
0,2860 NE Blossom Hill Rd,Fairview,97024,3.0,3.0,25.0,TOWNHOUSE,45.543571,1806.0,-122.441765,...,175.0,1950.0,273960.0,2020-01-01,https://www.zillow.com/homedetails/2860-NE-Blo...,2007-01-01,2860 NE Blossom Hill Rd,97024.0,NaN,"45.54357147, -122.4417648"
1,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,...,188.0,1995.0,397560.0,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01,1121 SW 10th Dr,97080.0,NaN,"45.48822784, -122.4444199"
2,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,...,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01,19309 NE Glisan St,97230.0,Rockwood,"45.52663422, -122.4640884"
3,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,...,351.0,1800.0,309260.0,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01,1518 SE 12th St,97080.0,NaN,"45.48799133, -122.4161835"
4,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,25.0,SINGLE_FAMILY,45.498184,2036.0,-122.451332,...,235.0,2250.0,373030.0,2020-01-01,https://www.zillow.com/homedetails/110-NW-Will...,1978-01-01,110 NW Willowbrook Ct,97030.0,NaN,"45.4981842, -122.4513321"


In [47]:
#rename some columns
zillow_positionstack_df=zillow_positionstack_df.rename(columns={"zipcode": "positionstack_zip", "address": "positionstack_address","neighborhood": "positionstack_neighborhood"})

### Zillow Google API DF Read In

In [38]:
# Store filepath in a variable
zillow_google = "Resources/cleaned_zillow_google_api.csv"

In [39]:
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
zillow_google_df = pd.read_csv(zillow_google, encoding="ISO-8859-1")
zillow_google_df.head()

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,...,taxAssessedYear,url,yearBuilt,house_number,street_name,google_zip,google_neighborhood,lat_lng,zipcode_length,google_address
0,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,...,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01,1131,Southwest Florence Drive,97080,Gresham Butte,"45.48822784, -122.4444199",5,1131 Southwest Florence Drive
1,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,...,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01,19309,Northeast Glisan Street,97230,North Gresham,"45.52663422, -122.4640884",5,19309 Northeast Glisan Street
2,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,...,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01,1518,Southeast 12th Street,97080,Asert,"45.48799133, -122.4161835",5,1518 Southeast 12th Street
3,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,25.0,SINGLE_FAMILY,45.498184,2036.0,-122.451332,...,2020-01-01,https://www.zillow.com/homedetails/110-NW-Will...,1978-01-01,110,Northwest Willowbrook Court,97030,Northwest,"45.4981842, -122.4513321",5,110 Northwest Willowbrook Court
4,3569 SW Mckinley St,Gresham,97080,3.0,3.0,41.0,SINGLE_FAMILY,45.475353,2209.0,-122.468307,...,2020-01-01,https://www.zillow.com/homedetails/3569-SW-Mck...,2017-01-01,3569,Southwest McKinley Street,97080,Pleasant Valley,"45.47535324, -122.4683075",5,3569 Southwest McKinley Street


# One Database with Multiple Tables

In [40]:
engine = create_engine('sqlite:///finalprojectdatabase.db', echo=True)
sqlite_connection = engine.connect()

In [41]:
sqlite_table = "arbnb"
ARBNB_df.to_sql(sqlite_table, sqlite_connection, if_exists='replace', index=False)

2022-04-20 13:52:31,513 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("arbnb")
2022-04-20 13:52:31,519 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:31,521 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("arbnb")
2022-04-20 13:52:31,521 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:31,522 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-04-20 13:52:31,522 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:31,523 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("arbnb")
2022-04-20 13:52:31,523 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:31,526 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-20 13:52:31,526 INFO sqlalchemy.engine.Engine [raw sql] ('arbnb',)
2022-04-20 13:52:31,527 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("arbnb")
2022-04-

In [42]:
sqlite_table1 = "arbnb_positionstack"
ARBNB_positionstack_api.to_sql(sqlite_table1, sqlite_connection, if_exists='replace', index=False)

2022-04-20 13:52:37,510 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("arbnb_positionstack")
2022-04-20 13:52:37,511 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:37,515 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("arbnb_positionstack")
2022-04-20 13:52:37,515 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:37,517 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-04-20 13:52:37,517 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:37,518 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("arbnb_positionstack")
2022-04-20 13:52:37,518 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:37,521 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-20 13:52:37,522 INFO sqlalchemy.engine.Engine [raw sql] ('arbnb_positionstack',)
2022-04-20 13:52:37,522 INFO sqlalchemy.engi

In [43]:
sqlite_table2 = "arbnb_google"
ARBNB_google_api.to_sql(sqlite_table2, sqlite_connection, if_exists='replace', index=False)

2022-04-20 13:52:39,188 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("arbnb_google")
2022-04-20 13:52:39,193 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:39,193 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("arbnb_google")
2022-04-20 13:52:39,194 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:39,195 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-04-20 13:52:39,195 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:39,196 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("arbnb_google")
2022-04-20 13:52:39,196 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:39,199 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-20 13:52:39,200 INFO sqlalchemy.engine.Engine [raw sql] ('arbnb_google',)
2022-04-20 13:52:39,200 INFO sqlalchemy.engine.Engine PRAGMA main.foreig

In [44]:
sqlite_table3 = "neighborhood"
neighborhood_df.to_sql(sqlite_table3, sqlite_connection, if_exists='replace', index=False)

2022-04-20 13:52:40,354 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("neighborhood")
2022-04-20 13:52:40,355 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:40,360 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("neighborhood")
2022-04-20 13:52:40,361 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:40,362 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-04-20 13:52:40,362 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:40,364 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("neighborhood")
2022-04-20 13:52:40,364 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:40,365 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-20 13:52:40,365 INFO sqlalchemy.engine.Engine [raw sql] ('neighborhood',)
2022-04-20 13:52:40,366 INFO sqlalchemy.engine.Engine PRAGMA main.foreig

In [45]:
sqlite_table4 = 'zillow'
zillow_df.to_sql(sqlite_table4, sqlite_connection, if_exists='replace', index=False)

2022-04-20 13:52:44,238 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("zillow")
2022-04-20 13:52:44,239 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:44,239 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("zillow")
2022-04-20 13:52:44,240 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:44,241 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-04-20 13:52:44,241 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:44,242 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("zillow")
2022-04-20 13:52:44,242 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:52:44,244 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-20 13:52:44,245 INFO sqlalchemy.engine.Engine [raw sql] ('zillow',)
2022-04-20 13:52:44,246 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("zillow")
202

In [48]:
sqlite_table5 = 'zillow_positionstack'
zillow_positionstack_df.to_sql(sqlite_table5, sqlite_connection, if_exists='replace', index=False)

2022-04-20 13:53:45,207 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("zillow_positionstack")
2022-04-20 13:53:45,207 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:53:45,212 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("zillow_positionstack")
2022-04-20 13:53:45,212 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:53:45,215 INFO sqlalchemy.engine.Engine 
CREATE TABLE zillow_positionstack (
	"Address" TEXT, 
	"City" TEXT, 
	"Zipcode" BIGINT, 
	bathrooms FLOAT, 
	bedrooms FLOAT, 
	"daysOnZillow" FLOAT, 
	"homeType" TEXT, 
	latitude FLOAT, 
	"livingArea" FLOAT, 
	longitude FLOAT, 
	"lotSize" FLOAT, 
	date_sold TEXT, 
	price BIGINT, 
	"pricePerSquareFoot" FLOAT, 
	"rentZestimate" FLOAT, 
	"taxAssessedValue" FLOAT, 
	"taxAssessedYear" TEXT, 
	url TEXT, 
	"yearBuilt" TEXT, 
	positionstack_address TEXT, 
	positionstack_zip FLOAT, 
	positionstack_neighborhood TEXT, 
	lat_lng TEXT
)


2022-04-20 13:53:45,216 INFO sqlalchemy.engine.Engine [no key 0.00065s] ()
20

In [49]:
sqlite_table6 = 'zillow_google'
zillow_google_df.to_sql(sqlite_table6, sqlite_connection, if_exists='replace', index=False)

2022-04-20 13:53:48,996 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("zillow_google")
2022-04-20 13:53:48,997 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:53:48,998 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("zillow_google")
2022-04-20 13:53:48,998 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 13:53:49,001 INFO sqlalchemy.engine.Engine 
CREATE TABLE zillow_google (
	"Address" TEXT, 
	"City" TEXT, 
	"Zipcode" BIGINT, 
	bathrooms FLOAT, 
	bedrooms FLOAT, 
	"daysOnZillow" FLOAT, 
	"homeType" TEXT, 
	latitude FLOAT, 
	"livingArea" FLOAT, 
	longitude FLOAT, 
	"lotSize" FLOAT, 
	date_sold TEXT, 
	price BIGINT, 
	"pricePerSquareFoot" FLOAT, 
	"rentZestimate" FLOAT, 
	"taxAssessedValue" FLOAT, 
	"taxAssessedYear" TEXT, 
	url TEXT, 
	"yearBuilt" TEXT, 
	house_number TEXT, 
	street_name TEXT, 
	google_zip BIGINT, 
	google_neighborhood TEXT, 
	lat_lng TEXT, 
	zipcode_length BIGINT, 
	google_address TEXT
)


2022-04-20 13:53:49,002 INFO sqlalchemy.engine.Engin

In [50]:
con = sqlite3.connect("finalprojectdatabase.db")
df = pd.read_sql_query("SELECT * from zillow_google", con)
df

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,...,taxAssessedYear,url,yearBuilt,house_number,street_name,google_zip,google_neighborhood,lat_lng,zipcode_length,google_address
0,1121 SW 10th Dr,Gresham,97080,3.0,4.0,11.0,SINGLE_FAMILY,45.488228,2724.0,-122.444420,...,2020-01-01,https://www.zillow.com/homedetails/1121-SW-10t...,1982-01-01,1131,Southwest Florence Drive,97080,Gresham Butte,"45.48822784, -122.4444199",5,1131 Southwest Florence Drive
1,19309 NE Glisan St,Portland,97230,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,...,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01,19309,Northeast Glisan Street,97230,North Gresham,"45.52663422, -122.4640884",5,19309 Northeast Glisan Street
2,1518 SE 12th St,Gresham,97080,2.0,3.0,14.0,SINGLE_FAMILY,45.487991,1150.0,-122.416184,...,2020-01-01,https://www.zillow.com/homedetails/1518-SE-12t...,1967-01-01,1518,Southeast 12th Street,97080,Asert,"45.48799133, -122.4161835",5,1518 Southeast 12th Street
3,110 NW Willowbrook Ct,Gresham,97030,2.0,3.0,25.0,SINGLE_FAMILY,45.498184,2036.0,-122.451332,...,2020-01-01,https://www.zillow.com/homedetails/110-NW-Will...,1978-01-01,110,Northwest Willowbrook Court,97030,Northwest,"45.4981842, -122.4513321",5,110 Northwest Willowbrook Court
4,3569 SW Mckinley St,Gresham,97080,3.0,3.0,41.0,SINGLE_FAMILY,45.475353,2209.0,-122.468307,...,2020-01-01,https://www.zillow.com/homedetails/3569-SW-Mck...,2017-01-01,3569,Southwest McKinley Street,97080,Pleasant Valley,"45.47535324, -122.4683075",5,3569 Southwest McKinley Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15347,15455 SW Gleneden Dr,Beaverton,97007,3.0,4.0,357.0,SINGLE_FAMILY,45.463707,2145.0,-122.835503,...,2019-01-01,https://www.zillow.com/homedetails/15455-SW-Gl...,1989-01-01,15455,Southwest Gleneden Drive,97007,Sexton Mountain,"45.46370697, -122.8355026",5,15455 Southwest Gleneden Drive
15348,7210 SW 156th Ave,Beaverton,97007,2.0,3.0,360.0,SINGLE_FAMILY,45.467113,1752.0,-122.837746,...,2019-01-01,https://www.zillow.com/homedetails/7210-SW-156...,1991-01-01,7210,Southwest 156th Avenue,97007,West Beaverton,"45.46711349, -122.8377457",5,7210 Southwest 156th Avenue
15349,7050 SW 152nd Ave,Beaverton,97007,2.0,3.0,360.0,SINGLE_FAMILY,45.468781,1924.0,-122.833855,...,2019-01-01,https://www.zillow.com/homedetails/7050-SW-152...,1981-01-01,7050,Southwest 152nd Avenue,97007,West Beaverton,"45.46878052, -122.8338547",5,7050 Southwest 152nd Avenue
15350,7600 SW Erica Pl,Beaverton,97008,2.0,3.0,363.0,SINGLE_FAMILY,45.465111,1378.0,-122.808701,...,2019-01-01,https://www.zillow.com/homedetails/7600-SW-Eri...,1977-01-01,7600,Southwest Erica Place,97008,Highland,"45.46511078, -122.8087006",5,7600 Southwest Erica Place


In [51]:
#merge tables zillow and neighborhood to get neighborhood name

# Connect to SQLite database
conn = sqlite3.connect("finalprojectdatabase.db")
  
# Create cursor object - needed to fetch the records from the database
cursor = conn.cursor()
  
# # Query for INNER JOIN
sql='''CREATE TABLE IF NOT EXISTS zillow_neighborhood AS
    SELECT zillow.Address, 
    zillow.City, 
    zillow.Zipcode, 
    neighborhood.Neighborhood_Name, 
    zillow.bathrooms, 
    zillow.bedrooms, 
    zillow.daysOnZillow, 
    zillow.homeType,  
    zillow.latitude,  
    zillow.livingArea, 
    zillow.longitude, 
    zillow.lotSize,  
    zillow.date_sold,  
    zillow.price,  
    zillow.pricePerSquareFoot,  
    zillow.rentZestimate,  
    zillow.taxAssessedValue,  
    zillow.taxAssessedYear,  
    zillow.url,  
    zillow.yearBuilt
FROM zillow 
INNER JOIN neighborhood
ON zillow.Zipcode = neighborhood.Zipcode;'''

# Executing the query - helps execute the query and return records according to the query
cursor.execute(sql)
  
# Fetching rows from the result table
# result = cursor.fetchall()
# for row in result:
#     print(row)

    
# # Closing the connection
conn.close()

In [52]:
#merge google api data inner join on address

# Connect to SQLite database
conn = sqlite3.connect("finalprojectdatabase.db")
  
# Create cursor object - needed to fetch the records from the database
cursor = conn.cursor()
  
# # Query for INNER JOIN
sql='''CREATE TABLE IF NOT EXISTS zillow_arbnb_google AS
    SELECT zillow_google.Address, 
    zillow_google.City, 
    zillow_google.Zipcode, 
    zillow_google.bathrooms, 
    zillow_google.bedrooms, 
    zillow_google.daysOnZillow, 
    zillow_google.homeType,  
    zillow_google.latitude,  
    zillow_google.livingArea, 
    zillow_google.longitude, 
    zillow_google.lotSize,  
    zillow_google.date_sold,  
    zillow_google.price,  
    zillow_google.pricePerSquareFoot,  
    zillow_google.rentZestimate,  
    zillow_google.taxAssessedValue,  
    zillow_google.taxAssessedYear,  
    zillow_google.url,  
    zillow_google.yearBuilt,
    zillow_google.google_address,
    zillow_google.google_zip,
    zillow_google.google_neighborhood,
    arbnb_google.listing_url,
    arbnb_google.host_response_time,
    arbnb_google.host_response_rate,
    arbnb_google.host_acceptance_rate,
    arbnb_google.host_is_superhost,
    arbnb_google.host_identity_verified,
    arbnb_google.neighbourhood_cleansed,
    arbnb_google.latitude,
    arbnb_google.longitude,
    arbnb_google.room_type,
    arbnb_google.accommodates,
    arbnb_google.bathrooms,
    arbnb_google.bedrooms,
    arbnb_google.beds,
    arbnb_google.price,
    arbnb_google.minimum_nights,
    arbnb_google.maximum_nights,
    arbnb_google.google_zip,
    arbnb_google.google_neighborhood,
    arbnb_google.lat_lng,
    arbnb_google.google_address    
FROM zillow_google
INNER JOIN arbnb_google
ON zillow_google.google_address = arbnb_google.google_address;'''

# Executing the query - helps execute the query and return records according to the query
cursor.execute(sql)

    
# # Closing the connection
conn.close()


In [53]:
con = sqlite3.connect("finalprojectdatabase.db")
zillow_arbnb_df = pd.read_sql_query("SELECT * from zillow_arbnb_google", con)
zillow_arbnb_df

,Address,City,Zipcode,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,...,bathrooms:1,bedrooms:1,beds,price:1,minimum_nights,maximum_nights,google_zip:1,google_neighborhood:1,lat_lng,google_address:1
0,804 SE 29th Ave,Portland,97214,2.0,4.0,354.0,SINGLE_FAMILY,45.516991,3300.0,-122.635773,...,1.0,1.0,2.0,65.0,1,1125,97214,Sunnyside,"45.51709, -122.63582",804 Southeast 29th Avenue
1,6125 SW 47th Ave,Portland,97221,2.0,4.0,42.0,SINGLE_FAMILY,45.479443,2570.0,-122.725487,...,1.0,1.0,1.0,75.0,1,14,97221,Hayhurst,"45.47952, -122.72538",6125 Southwest 47th Avenue
2,1337 SE 15th Ave #A,Portland,97214,1.0,4.0,32.0,SINGLE_FAMILY,45.513065,1975.0,-122.650841,...,1.0,1.0,3.0,81.0,2,29,97214,Buckman Neighborhood,"45.51296, -122.65083",1337 Southeast 15th Avenue
3,3636 SE Morrison St,Portland,97214,2.0,3.0,70.0,SINGLE_FAMILY,45.516941,2231.0,-122.626404,...,1.5,1.0,1.0,116.0,2,90,97214,Sunnyside,"45.51691, -122.6265",3636 Southeast Morrison Street
4,835 N Webster St,Portland,97217,2.0,4.0,102.0,SINGLE_FAMILY,45.559940,2808.0,-122.675797,...,1.0,2.0,2.0,124.0,2,120,97217,North Portland,"45.55977, -122.67575",835 North Webster Street
5,6428 SW Burlingame Pl,Portland,97239,3.0,5.0,149.0,SINGLE_FAMILY,45.478409,2750.0,-122.686188,...,1.0,1.0,1.0,74.0,1,22,97239,Southwest Portland,"45.47822, -122.68605",6428 Southwest Burlingame Place
6,5406 SE 43rd Ave,Portland,97206,4.0,4.0,119.0,SINGLE_FAMILY,45.483940,3602.0,-122.618317,...,1.0,1.0,1.0,65.0,30,1125,97206,Southeast Portland,"45.48391, -122.61835",5406 Southeast 43rd Avenue
7,1023 SW Rivington Dr,Portland,97201,3.0,2.0,114.0,SINGLE_FAMILY,45.508217,2543.0,-122.690002,...,1.0,1.0,1.0,82.0,1,7,97201,Southwest Hills,"45.5079, -122.69002",1023 Southwest Rivington Drive
8,2604 SE 32nd Ave,Portland,97202,2.0,4.0,84.0,SINGLE_FAMILY,45.503960,3434.0,-122.632240,...,1.0,1.0,3.0,73.0,2,28,97202,Richmond,"45.50392, -122.63253",2604 Southeast 32nd Avenue
9,3367 NE Holladay St,Portland,97232,2.0,3.0,212.0,SINGLE_FAMILY,45.530418,2910.0,-122.629280,...,1.0,1.0,2.0,78.0,2,4,97232,Laurelhurst,"45.53032, -122.62936",3367 Northeast Holladay Street


In [54]:
#export zillow to csv
zillow_arbnb_df.to_csv('Resources/merged_zillow_arbnb.csv',index=False)

In [202]:
con = sqlite3.connect("finalprojectdatabase.db")
df2 = pd.read_sql_query("SELECT * from zillow_neighborhood", con)
df2

,Address,City,Zipcode,Neighborhood_Name,bathrooms,bedrooms,daysOnZillow,homeType,latitude,livingArea,longitude,lotSize,date_sold,price,pricePerSquareFoot,rentZestimate,taxAssessedValue,taxAssessedYear,url,yearBuilt
0,19309 NE Glisan St,Portland,97230,Argay,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
1,19309 NE Glisan St,Portland,97230,Glenfair,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
2,19309 NE Glisan St,Portland,97230,Hazelwood,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
3,19309 NE Glisan St,Portland,97230,Russell,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
4,19309 NE Glisan St,Portland,97230,Wilkes,1.0,3.0,11.0,SINGLE_FAMILY,45.526634,1217.0,-122.464088,13939.0,2021-06-04,348000,286.0,1695.0,269520.0,2017-01-01,https://www.zillow.com/homedetails/19309-NE-Gl...,1953-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39445,216 NW 117th Ave,Portland,97229,Northwest Heights,2.5,4.0,297.0,SINGLE_FAMILY,45.520996,2035.0,-122.797989,2614.0,2020-09-25,540000,265.0,2627.0,549020.0,2019-01-01,https://www.zillow.com/homedetails/216-NW-117t...,2012-01-01
39446,219 NW 117th Loop,Beaverton,97229,Forest Park,3.0,3.0,193.0,SINGLE_FAMILY,45.520420,1619.0,-122.798737,2178.0,2021-01-07,490000,303.0,2462.0,456670.0,2019-01-01,https://www.zillow.com/homedetails/219-NW-117t...,2012-01-01
39447,219 NW 117th Loop,Beaverton,97229,Northwest Heights,3.0,3.0,193.0,SINGLE_FAMILY,45.520420,1619.0,-122.798737,2178.0,2021-01-07,490000,303.0,2462.0,456670.0,2019-01-01,https://www.zillow.com/homedetails/219-NW-117t...,2012-01-01
39448,712 NW 118th Ave,Portland,97229,Forest Park,2.5,2.0,164.0,TOWNHOUSE,45.525494,1129.0,-122.796677,435.0,2021-02-05,358000,317.0,1864.0,317820.0,2019-01-01,https://www.zillow.com/homedetails/712-NW-118t...,2006-01-01


In [ ]:
# #merge tables zillow and arbnb tables

# # con = sqlite3.connect("finalprojectdatabase.db")
# # df2 = pd.read_sql_query("SELECT zillow.Address, zillow.City, zillow.Zipcode, neighborhood.Neighborhood_Name, zillow.bathrooms, zillow.bedrooms, zillow.daysOnZillow, zillow.homeType, zillow.latitude, zillow.livingArea,zillow.longitude, zillow.lotSize, zillow.date_sold, zillow.price, zillow.priceHistory/0/pricePerSquareFoot, zillow.rentZestimate, zillow.taxAssessedValue, zillow.taxAssessedYear, zillow.url, zillow.yearBuilt FROM zillow INNER JOIN neighborhood ON zillow.Zipcode = neighborhood.Zip_Code;", con)
# # df2

 
# # Connect to SQLite database
# conn = sqlite3.connect("finalprojectdatabase.db")
  
# # Create cursor object - needed to fetch the records from the database
# cursor = conn.cursor()
  
# # # Query for INNER JOIN
# sql='''CREATE TABLE zillow_arbnb AS
#     SELECT * 
#     FROM zillow 
#     FULL OUTER JOIN arbnb 
#     ON zillow.column_name=arbnb.column_name;
# ON zillow.Zipcode = neighborhood.Zipcode;'''

# # Executing the query - helps execute the query and return records according to the query
# cursor.execute(sql)
  
# # Fetching rows from the result table
# # result = cursor.fetchall()
# # for row in result:
# #     print(row)

    
# # # Closing the connection
# conn.close()
